<a href="https://colab.research.google.com/github/G0nkly/pytorch_sandbox/blob/main/gpts/AK_GPT_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

In [2]:
# consider the following example

torch.manual_seed(1337)
B, T, C = 4, 8, 2 # batch, time, channels
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 8, 2])

In [3]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B, T, C))
for b in range(B):
  for t in range(T):
    xprev = x[b, :t+1] # (t, C)
    print(xprev)
    xbow[b,t] = torch.mean(xprev, 0)

tensor([[ 0.1808, -0.0700]])
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152]])
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255]])
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643]])
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679]])
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102]])
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398]])
tensor([[ 0.1808, -0.0700],
        [-0.3596, -0.9152],
        [ 0.6258,  0.0255],
        [ 0.9545,  0.0643],
        [ 0.3612,  1.1679],
        [-1.3499, -0.5102],
        [ 0.2360, -0.2398],
        [-0.9

In [12]:
t = torch.randn(4, 8)
print(t)
torch.mean(t, 1)

tensor([[ 2.2874,  0.9611, -1.5297, -0.2912, -0.1140, -0.3137, -0.6293,  1.1385],
        [-0.9913,  0.1700,  1.2249, -0.2345, -1.0572, -0.6543,  1.5909, -0.6995],
        [-0.8961,  0.0662, -0.0563,  2.3412, -2.7234,  0.5097, -0.8145, -0.2460],
        [ 0.0045,  2.0474, -0.1575, -0.2187, -1.3519, -0.0573, -1.8540, -1.3849]])


tensor([ 0.1887, -0.0814, -0.2274, -0.3715])